<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%203/NLP_09_Mineri%CC%81a_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minería de texto
Vamos a ver distintas técnicas de extracción de la información contenida en el texto, para convertirla en inf. estructurada.

In [ ]:
import spacy
import pandas as pd
from spacy import displacy

nlp = spacy.load("es_core_news_sm")

## Named Entity Recognition

In [ ]:
doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
entidades = [e for e in doc.ents]
entidades

[Miguel de Cervantes, Alcalá de Henares]

Cada entidad es un `span` de texto marcado con una etiqueta en los *tokens* (esquema `IOB`) y un tipo de entidad (propiedad `label` del `span`)

In [ ]:
type(entidades[0])

spacy.tokens.span.Span

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,El,DET,O,
1,gran,ADJ,O,
2,escritor,NOUN,O,
3,Miguel,PROPN,B,PER
4,de,ADP,I,PER
5,Cervantes,PROPN,I,PER
6,nació,VERB,O,
7,en,ADP,O,
8,Alcalá,PROPN,B,LOC
9,de,ADP,I,LOC


In [ ]:
doc[entidades[0].start:entidades[0].end]

Miguel de Cervantes

In [ ]:
entidades[0].label_

'PER'

### Creación de nuevas entidades en spaCy
Podemos crear entidades nuevas con el componente `EntityRuler` del pipeline (https://spacy.io/api/entityruler)

In [ ]:
#Definimos "escritor" como profesión
#Nota: con el regex que se ha añadido, se detectará tanto 'escritor' como 'escritora'
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "PROFESION", "pattern": [{"LOWER": {"REGEX": "escritora?"}}]}]
ruler.add_patterns(patterns)

doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")
print([(ent.text, ent.label_) for ent in doc.ents])

ValueError: [E007] 'entity_ruler' already exists in pipeline. Existing names: ['tok2vec', 'morphologizer', 'parser', 'senter', 'ner', 'attribute_ruler', 'lemmatizer', 'entity_ruler']

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,El,DET,O,
1,gran,ADJ,O,
2,escritor,NOUN,B,PROFESION
3,Miguel,PROPN,B,PER
4,de,ADP,I,PER
5,Cervantes,PROPN,I,PER
6,nació,VERB,O,
7,en,ADP,O,
8,Alcalá,PROPN,B,LOC
9,de,ADP,I,LOC


Tal y como se puede comprobar en la tabla de arriba, el ent_iob sirve para ver dónde comienza y acaba cada entidad. Por ejemplo, Miguel de Cervantes lo detecta como una única entidad:
- **I**: Token is **inside** an entity
- **O**: Token is **outside** an entity
- **B**: Token is at the **beginning** of an entity

Podemos añadir nuevas profesiones

In [ ]:
patterns = [{"label": "PROFESION", "pattern": [{"LOWER": "matador"},{"LOWER": "de"},{"LOWER": "toros"}]}]
ruler.add_patterns(patterns)

doc = nlp("El padre de Miguel Bosé era el matador de toros Luis Miguel Dominguín")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Miguel Bosé', 'PER'), ('matador de toros', 'PROFESION'), ('Luis Miguel Dominguín', 'PER')]


In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,El,DET,O,
1,padre,NOUN,O,
2,de,ADP,O,
3,Miguel,PROPN,B,PER
4,Bosé,PROPN,I,PER
5,era,AUX,O,
6,el,DET,O,
7,matador,NOUN,B,PROFESION
8,de,ADP,I,PROFESION
9,toros,NOUN,I,PROFESION


In [ ]:
doc = nlp("Luis Miguel era matador de toros y Rosalía de Castro era escritora")
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,ent_iob,ent_type
0,Luis,PROPN,B,PER
1,Miguel,PROPN,I,PER
2,era,AUX,O,
3,matador,NOUN,B,PROFESION
4,de,ADP,I,PROFESION
5,toros,NOUN,I,PROFESION
6,y,CCONJ,O,
7,Rosalía,PROPN,B,PER
8,de,ADP,I,PER
9,Castro,PROPN,I,PER


In [ ]:
patterns = [{"label": "ANIMAL", "pattern": [{"LEMMA": "gato"},{"LEMMA": "montés", 'OP': '?'}]},
           {"label": "ANIMAL", "pattern": [{"LEMMA": "perro"}]}]
ruler.add_patterns(patterns)

doc = nlp("Gatos y perros son animales de compañía.")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Gatos', 'ANIMAL'), ('perros', 'ANIMAL')]


In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'lemma': t.lemma_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,lemma,ent_iob,ent_type
0,Gatos,NOUN,gato,B,ANIMAL
1,y,CCONJ,y,O,
2,perros,NOUN,perro,B,ANIMAL
3,son,AUX,ser,O,
4,animales,NOUN,animal,O,
5,de,ADP,de,O,
6,compañía,NOUN,compañía,O,
7,.,PUNCT,.,O,


In [ ]:
doc = nlp("El gato montés es más grande que los gatos callejeros")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'lemma': t.lemma_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

,token,POS,lemma,ent_iob,ent_type
0,El,DET,el,O,
1,gato,NOUN,gato,B,ANIMAL
2,montés,PROPN,montés,I,ANIMAL
3,es,AUX,ser,O,
4,más,ADV,más,O,
5,grande,ADJ,grande,O,
6,que,SCONJ,que,O,
7,los,DET,el,O,
8,gatos,NOUN,gato,B,ANIMAL
9,callejeros,ADJ,callejero,O,


In [ ]:
print([(ent.text, ent.label_) for ent in doc.ents])

[('gato montés', 'ANIMAL'), ('gatos', 'ANIMAL')]


In [ ]:
doc = nlp("El escritor Miguel de Cervantes tenía un gato llamado Juan")
print([(ent.text, ent.label_) for ent in doc.ents])

[('escritor', 'PROFESION'), ('Miguel de Cervantes', 'PER'), ('gato', 'ANIMAL'), ('Juan', 'PER')]


## Entity linking
Usamos la API de DBPedia Spotlight (https://www.dbpedia-spotlight.org/api) para relacionar entidades propias del texto con entradas en la BBDD de conocomientos [dbpedia](http://dbpedia.org).

In [ ]:
import requests
from IPython.core.display import display, HTML
# An API Error Exception
class APIError(Exception):
    def __init__(self, status):
            self.status = status
    def __str__(self):
            return "APIError: status={}".format(self.status)
      
# Base URL for Spotlight API
base_url = "http://api.dbpedia-spotlight.org/en/annotate"
# Parameters 
# 'text' - text to be annotated 
# 'confidence' -   confidence score for linking
text = "The Space Shuttle was a partially reusable low Earth orbital spacecraft system operated from April 12, 1981, to July 21, 2011, by the National Aeronautics and Space Administration in the United States. Launched from the Kennedy Space Center in Florida, five Space Shuttle orbiter vehicles flew on a total of 135 missions during 30 years."

params = {"text": text, "confidence": 0.5}
# Response content type
headers = {'accept': 'text/html'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
display(HTML(res.text))

Tal y como se puede comprobar, con un umbral de **0.5** cosas como la NASA (National Aeronautics and Space Administration) no lo detect como entidad.
Subir el umbral hace que se detecten menos entidades, y bajar el umbral hace que se detecten más
Por ejemplo, probemos con **0.7** y con **0.3**:

In [ ]:
params = {"text": text, "confidence": 0.7}
# Response content type
headers = {'accept': 'text/html'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
display(HTML(res.text))

In [ ]:
params = {"text": text, "confidence": 0.3}
# Response content type
headers = {'accept': 'text/html'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
display(HTML(res.text))

In [ ]:
res.headers

{'Date': 'Fri, 07 May 2021 17:18:25 GMT', 'Server': 'grizzly/1.9.48', 'Access-Control-Allow-Origin': '*', 'Content-Type': 'text/html', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '814', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive'}

In [ ]:
res.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n<head>\n<title>DBpedia Spotlight annotation</title>\n<meta http-equiv="Content-type" content="text/html;charset=UTF-8">\n</head>\n<body>\n<div>\nThe <a href="http://dbpedia.org/resource/Space_Shuttle" title="http://dbpedia.org/resource/Space_Shuttle" target="_blank">Space Shuttle</a> was a <a href="http://dbpedia.org/resource/Reusable_launch_system" title="http://dbpedia.org/resource/Reusable_launch_system" target="_blank">partially reusable</a> low <a href="http://dbpedia.org/resource/Earth" title="http://dbpedia.org/resource/Earth" target="_blank">Earth</a> <a href="http://dbpedia.org/resource/Orbital_spaceflight" title="http://dbpedia.org/resource/Orbital_spaceflight" target="_blank">orbital</a> <a href="http://dbpedia.org/resource/Spacecraft" title="http://dbpedia.org/resource/Spacecraft" target="_blank">spacecraft</a> <a href="http://dbpedia.org/resource/System" title="

Podemos hacer la petición en formato JSON, lo que devuelve más información para cada entidad detectada:

In [ ]:
headers = {'accept': 'application/json'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
respuesta = res.json()
respuesta

{'@text': 'The Space Shuttle was a partially reusable low Earth orbital spacecraft system operated from April 12, 1981, to July 21, 2011, by the National Aeronautics and Space Administration in the United States. Launched from the Kennedy Space Center in Florida, five Space Shuttle orbiter vehicles flew on a total of 135 missions during 30 years.',
 '@confidence': '0.3',
 '@support': '0',
 '@types': '',
 '@sparql': '',
 '@policy': 'whitelist',
 'Resources': [{'@URI': 'http://dbpedia.org/resource/Space_Shuttle',
   '@support': '5777',
   '@types': 'Wikidata:Q41291,DBpedia:MeanOfTransportation,DBpedia:Rocket',
   '@surfaceForm': 'Space Shuttle',
   '@offset': '4',
   '@similarityScore': '0.8328158026333504',
   '@percentageOfSecondRank': '0.20065367572673085'},
  {'@URI': 'http://dbpedia.org/resource/Reusable_launch_system',
   '@support': '411',
   '@types': '',
   '@surfaceForm': 'partially reusable',
   '@offset': '24',
   '@similarityScore': '1.0',
   '@percentageOfSecondRank': '0.0'

La respuesta JSON se puede iterar como un diccionario de claves-valores:

In [ ]:
for key, value in respuesta.items():
    print(f"{key} : {value}\n")

@text : The Space Shuttle was a partially reusable low Earth orbital spacecraft system operated from April 12, 1981, to July 21, 2011, by the National Aeronautics and Space Administration in the United States. Launched from the Kennedy Space Center in Florida, five Space Shuttle orbiter vehicles flew on a total of 135 missions during 30 years.

@confidence : 0.3

@support : 0

@types : 

@sparql : 

@policy : whitelist

Resources : [{'@URI': 'http://dbpedia.org/resource/Space_Shuttle', '@support': '5777', '@types': 'Wikidata:Q41291,DBpedia:MeanOfTransportation,DBpedia:Rocket', '@surfaceForm': 'Space Shuttle', '@offset': '4', '@similarityScore': '0.8328158026333504', '@percentageOfSecondRank': '0.20065367572673085'}, {'@URI': 'http://dbpedia.org/resource/Reusable_launch_system', '@support': '411', '@types': '', '@surfaceForm': 'partially reusable', '@offset': '24', '@similarityScore': '1.0', '@percentageOfSecondRank': '0.0'}, {'@URI': 'http://dbpedia.org/resource/Earth', '@support': '39

Además podemos acceder directamente a una clave o a una clave anidada:

In [ ]:
respuesta['@text']

'The Space Shuttle was a partially reusable low Earth orbital spacecraft system operated from April 12, 1981, to July 21, 2011, by the National Aeronautics and Space Administration in the United States. Launched from the Kennedy Space Center in Florida, five Space Shuttle orbiter vehicles flew on a total of 135 missions during 30 years.'

In [ ]:
type(respuesta['Resources'])

list

In [ ]:
respuesta['Resources'][0]['@URI']

'http://dbpedia.org/resource/Space_Shuttle'

# Extracción de palabras clave
Usamos la librería `textacy` sobre `spaCy` (https://textacy.readthedocs.io/en/latest/). \
Instalamos con:
```
conda install -c conda-forge textacy
```

In [ ]:
with open('articulo.txt', 'r') as f:
    texto = f.read()
    
texto

'La Policía Nacional, en colaboración con la policía marroquí, ha desarticulado una "importante y peligrosa" célula del Estado Islámico (ISIS, en sus siglas en inglés) que pretendía impulsar atentados yihadistas en España y en otros países de Europa: "Tenía como objetivo materializar la estrategia de la organización terrorista en occidente", aseguran fuentes de la investigación, que aseguran que estaban en contacto con yihadistas ubicados en Siria. Los agentes de la Comisaría General de Información (CGI) y los de la Dirección General de Vigilancia del Territorio del Reino de Marruecos (DGST) han detenido en total a cinco personas, dos en España, en las localidades de Lorca (Murcia) y Abadiño (Bizkaia); y otras tres en Marruecos, que integraban la red de la organización terrorista.'

In [ ]:
doc = nlp(texto)

In [ ]:
len(doc)

148

In [ ]:
from textacy.extract import keyterms as kt
#https://textacy.readthedocs.io/en/latest/api_reference/extract.html#keyterms

In [ ]:
kt.textrank(doc, normalize="lemma", topn=10)

[('atentado yihadista', 0.024277581688875352),
 ('yihadista ubicado', 0.022993375155304988),
 ('organización terrorista', 0.019293287969168793),
 ('policía marroquí', 0.01655092427910999),
 ('Policía Nacional', 0.01655092427910999),
 ('Marruecos', 0.013426001804612698),
 ('España', 0.010335310756957202),
 ('objetivo', 0.010071860790802918),
 ('persona', 0.008953298332459406),
 ('Islámico', 0.008799611873242349)]

In [ ]:
kt.textrank(doc, normalize="lower", topn=10)

[('atentados yihadistas', 0.023920483241090973),
 ('yihadistas ubicados', 0.022783778625400142),
 ('policía nacional', 0.019458089819428884),
 ('policía marroquí', 0.019458089819428884),
 ('organización terrorista', 0.01927156573993427),
 ('marruecos', 0.013310520633285614),
 ('españa', 0.010349544701617894),
 ('objetivo', 0.009858232010108544),
 ('personas', 0.008780788680023245),
 ('islámico', 0.008563872991708348)]

In [ ]:
kt.sgrank(doc, normalize="lower", topn=0.1)

[('organización terrorista', 0.460942936743379),
 ('yihadistas ubicados', 0.19821659390486374),
 ('policía marroquí', 0.04730287324545998),
 ('atentados yihadistas', 0.046543802059051204),
 ('policía nacional', 0.037396076179293546)]

In [ ]:
#con un texto más largo
with open('cañas y barro.txt', 'r') as f:
    texto = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'cañas y barro.txt'

In [ ]:
doc = nlp(texto)

In [ ]:
kt.sgrank(doc, normalize="lower", topn=10)